### Treap y árboles de intervalos

Un treap es un árbol binario de búsqueda combinado con las propiedades de un heap, por lo que cada nodo posee una clave (key) y una prioridad asignada al azar. La clase `TreapNode` define un nodo del treap, almacenando la clave, una prioridad aleatoria, punteros a subárboles izquierdo y derecho, y un contador del tamaño del subárbol. Se provee además el método `update()` para recalcular dicho tamaño. Las funciones `split` y `merge` son esenciales: `split` divide el treap en dos árboles basados en un valor clave dado, mientras que merge une dos treaps respetando la propiedad de heap basada en la prioridad.

La clase `Treap`, que encapsula la funcionalidad del treap, ofrece operaciones de inserción, búsqueda, eliminación y obtención del k-ésimo elemento en el recorrido inorden. En `insert()` se verifica la existencia de la clave, eliminándola de haberla encontrado, e inserta el nuevo nodo combinando los subárboles divididos. La función `kth` permite encontrar el nodo que ocupa la k-ésima posición del recorrido inorden.

Un **árbol de intervalos** es una estructura de datos que almacena intervalos (por ejemplo, rangos numéricos o temporales) y permite consultar, de forma eficiente, qué intervalos se superponen con uno dado. Cada nodo almacena un intervalo y el máximo extremo superior de su subárbol, facilitando la búsqueda de solapamientos. Los treaps, que combinan las propiedades de árboles binarios de búsqueda y heaps, ofrecen balanceo probabilístico y operaciones de inserción, eliminación y búsqueda eficientes. Al usarlos como base para implementar árboles de intervalos, se optimiza el rendimiento y se mejora la gestión de consultas en colecciones de intervalos en aplicaciones críticas

Se implementa un **árbol de intervalos** mediante las clases `IntervalNode` e `IntervalTree`. Cada nodo de intervalo almacena un rango `[low, high]`, junto a un valor max que representa el mayor límite superior en su subárbol y punteros a hijos izquierdo y derecho. La actualización de max se realiza con el método `update_max()`. En `IntervalTree`, la función `_insert` inserta un nodo basado en el valor `low` y actualiza el valor `max` en cada paso. La búsqueda de intersecciones se lleva a cabo con `_search_overlap`, acumulando en una lista los intervalos que se solapan con la consulta.

Se incluye pruebas que demuestran inserción, eliminación, búsqueda y recorridos inorden en el treap y el árbol de intervalos, mostrando correcto funcionamiento.

In [ ]:
import random

#Implementación del treap     

class TreapNode:
    """
    Nodo de un Treap.
    Cada nodo contiene:
        - key: La clave numérica del nodo.
        - priority: Una prioridad aleatoria (para mantener la propiedad de heap).
        - left: Hijo izquierdo.
        - right: Hijo derecho.
        - size: Tamaño del subárbol con raíz en este nodo (útil para operaciones de orden).
        - value: Valor asociado (opcional, en caso de que se quiera almacenar información extra).
    """
    def __init__(self, key, value=None):
        self.key = key
        self.priority = random.random()  # Prioridad aleatoria
        self.left = None
        self.right = None
        self.size = 1  # Inicialmente, solo el nodo mismo
        self.value = value

    def update(self):
        """Actualiza el tamaño del subárbol basándose en los tamaños de sus hijos."""
        self.size = 1
        if self.left is not None:
            self.size += self.left.size
        if self.right is not None:
            self.size += self.right.size

def split(root, key):
    """
    Divide el treap 'root' en dos treaps:
    - Izquierdo con claves < key.
    - Derecho con claves >= key.
    """
    if root is None:
        return (None, None)
    if key <= root.key:
        left_subtree, root.left = split(root.left, key)
        root.update()
        return (left_subtree, root)
    else:
        root.right, right_subtree = split(root.right, key)
        root.update()
        return (root, right_subtree)

def merge(left, right):
    """
    Combina dos treaps 'left' y 'right' suponiendo que todas las claves en 'left'
    son menores o iguales que las de 'right'.
    """
    if left is None or right is None:
        return left or right
    if left.priority < right.priority:
        left.right = merge(left.right, right)
        left.update()
        return left
    else:
        right.left = merge(left, right.left)
        right.update()
        return right

class Treap:
    """
    Clase que encapsula el Treap.
    Permite insertar, eliminar, buscar un elemento, obtener el k-ésimo elemento y recorrer en orden.
    """
    def __init__(self):
        self.root = None

    def insert(self, key, value=None):
        """Inserta un nodo con la clave 'key' y valor opcional 'value'."""
        # Si ya existe, podemos eliminar el nodo para insertar uno nuevo
        if self.search(key) is not None:
            self.delete(key)
        new_node = TreapNode(key, value)
        left_subtree, right_subtree = split(self.root, key)
        merged = merge(left_subtree, new_node)
        self.root = merge(merged, right_subtree)

    def search(self, key):
        """Busca el nodo con la clave 'key'. Retorna el nodo si existe, o None en caso contrario."""
        node = self.root
        while node is not None:
            if key == node.key:
                return node
            elif key < node.key:
                node = node.left
            else:
                node = node.right
        return None

    def _delete(self, root, key):
        """Elimina de forma recursiva un nodo con la clave 'key' a partir del nodo 'root'."""
        if root is None:
            return None
        if root.key == key:
            # Combina sus dos hijos para mantener la propiedad del treap
            return merge(root.left, root.right)
        elif key < root.key:
            root.left = self._delete(root.left, key)
            root.update()
        else:
            root.right = self._delete(root.right, key)
            root.update()
        return root

    def delete(self, key):
        """Elimina el nodo con la clave 'key' del treap."""
        self.root = self._delete(self.root, key)

    def kth(self, k):
        """
        Retorna el k-ésimo nodo en el recorrido en orden (k-ésimo menor).
        Si k es inválido, retorna None.
        """
        if self.root is None or k <= 0 or k > self.root.size:
            return None
        node = self.root
        while node is not None:
            left_size = node.left.size if node.left is not None else 0
            if k == left_size + 1:
                return node
            elif k <= left_size:
                node = node.left
            else:
                k -= left_size + 1
                node = node.right
        return None

    def inorder(self):
        """
        Retorna una lista con las claves del treap en orden creciente.
        Útil para verificar la estructura del árbol.
        """
        result = []
        def dfs(node):
            if node:
                dfs(node.left)
                result.append(node.key)
                dfs(node.right)
        dfs(self.root)
        return result

    def __str__(self):
        """Representación en cadena (inorden) del Treap."""
        return "Treap(inorder): " + str(self.inorder())

#Implementación del árbol de intervalos
class IntervalNode:
    """
    Nodo de un árbol de intervalos.
    Cada nodo almacena un intervalo [low, high] y mantiene:
        - low, high: Los extremos del intervalo.
        - max: El máximo valor 'high' en el subárbol con raíz en este nodo.
        - left: Hijo izquierdo.
        - right: Hijo derecho.
    """
    def __init__(self, low, high):
        self.low = low
        self.high = high
        self.max = high  # Inicialmente, es el propio high
        self.left = None
        self.right = None

    def update_max(self):
        """
        Actualiza el valor 'max' basándose en el máximo de 'high' de los hijos.
        """
        self.max = self.high
        if self.left is not None:
            self.max = max(self.max, self.left.max)
        if self.right is not None:
            self.max = max(self.max, self.right.max)

class IntervalTree:
    """
    Árbol de Intervalos que permite insertar intervalos y buscar todos aquellos que se
    solapan con un intervalo dado.
    """
    def __init__(self):
        self.root = None

    def _insert(self, root, node):
        """
        Inserta un nodo 'node' en el subárbol con raíz 'root' siguiendo la clave 'low'.
        Se actualiza la información 'max' en cada paso.
        """
        if root is None:
            return node
        if node.low < root.low:
            root.left = self._insert(root.left, node)
        else:
            root.right = self._insert(root.right, node)
        root.update_max()
        return root

    def insert(self, low, high):
        """Inserta un intervalo [low, high] en el árbol."""
        new_node = IntervalNode(low, high)
        self.root = self._insert(self.root, new_node)

    def _search_overlap(self, node, interval, result):
        """
        Búsqueda recursiva de intervalos que se solapan con el intervalo dado.
        'interval' es una tupla (low, high) y 'result' es la lista donde se acumulan los intervalos que se solapan.
        """
        if node is None:
            return
        # Si el intervalo de este nodo se solapa con el intervalo de consulta
        if interval[0] <= node.high and interval[1] >= node.low:
            result.append((node.low, node.high))
        # Si existe un hijo izquierdo y su valor máximo es mayor o igual que el límite inferior de consulta,
        # puede haber solapamientos en el subárbol izquierdo.
        if node.left is not None and node.left.max >= interval[0]:
            self._search_overlap(node.left, interval, result)
        # Siempre se continúa a la derecha
        self._search_overlap(node.right, interval, result)

    def search_overlaps(self, low, high):
        """
        Retorna una lista de todos los intervalos del árbol que se solapan con
        el intervalo [low, high].
        """
        result = []
        self._search_overlap(self.root, (low, high), result)
        return result

    def _inorder(self, node, result):
        """
        Recorrido inorden para obtener la lista de intervalos
        junto con el valor 'max' actual de cada nodo.
        """
        if node is not None:
            self._inorder(node.left, result)
            result.append((node.low, node.high, node.max))
            self._inorder(node.right, result)

    def inorder(self):
        """
        Retorna una lista con los intervalos del árbol en orden (por el valor 'low').
        Cada elemento es una tupla: (low, high, max).
        """
        result = []
        self._inorder(self.root, result)
        return result

    def __str__(self):
        """Representación en cadena del árbol de intervalos."""
        return "IntervalTree(inorder): " + str(self.inorder())


# Ejemplos y pruebas de código

# Pruebas con el treap
print("Pruebas del treap")
treap = Treap()
# Inserción de claves
claves = [50, 30, 70, 20, 40, 60, 80]
for clave in claves:
    treap.insert(clave)
    print(f"Inserción de {clave}: {treap}")

# Búsqueda de un nodo
clave_buscar = 40
nodo = treap.search(clave_buscar)
if nodo:
    print(f"El nodo con clave {clave_buscar} fue encontrado con prioridad {nodo.priority:.4f} y tamaño {nodo.size}.")
else:
    print(f"El nodo con clave {clave_buscar} no fue encontrado.")

# Encontrar el k-ésimo elemento (por ejemplo el 3er elemento)
k = 3
nodo_k = treap.kth(k)
if nodo_k:
    print(f"El {k}-ésimo elemento es {nodo_k.key}.")
else:
    print(f"No se pudo encontrar el {k}-ésimo elemento.")

# Eliminación de un nodo
clave_eliminar = 70
treap.delete(clave_eliminar)
print(f"Después de eliminar {clave_eliminar}: {treap}")
print("Recorrido inorden final del treap:", treap.inorder())

print("\n")

# Pruebas con el árbol de intervalos
print("Pruebas del árbol de intervalos")
interval_tree = IntervalTree()
# Inserción de intervalos
intervalos = [(15, 20), (10, 30), (17, 19), (5, 20), (12, 15), (30, 40)]
for low, high in intervalos:
    interval_tree.insert(low, high)
    print(f"Inserción del intervalo [{low}, {high}]: {interval_tree}")

# Mostrar el recorrido inorden del árbol (para verificar la información de 'max')
print("Recorrido inorden del árbol de intervalos:")
for elem in interval_tree.inorder():
    print(elem)

# Consulta de intersección: Buscar intervalos que se solapan con [14, 16]
consulta = (14, 16)
solapamientos = interval_tree.search_overlaps(consulta[0], consulta[1])
print(f"\nIntervalos que se solapan con {consulta}:")
for inter in solapamientos:
    print(inter)

# Consulta adicional: Buscar intervalos que se solapan con [21, 23]
consulta2 = (21, 23)
solapamientos2 = interval_tree.search_overlaps(consulta2[0], consulta2[1])
print(f"\nIntervalos que se solapan con {consulta2}: {solapamientos2}")

# Funciones adicionales de prueba

def demo_treap_operations():
    """
    Demostración adicional de operaciones sobre treap.
    Inserta, elimina, y obtiene el k-ésimo elemento mostrando el estado tras cada operación.
    """
    print("\n--- Demostración de operaciones en treap ---")
    demo_treap = Treap()
    operaciones = [25, 10, 35, 5, 15, 30, 40]
    for op in operaciones:
        demo_treap.insert(op)
        print(f"Después de insertar {op}: {demo_treap.inorder()}")
    demo_treap.delete(15)
    print("Después de eliminar 15:", demo_treap.inorder())
    for k in range(1, demo_treap.root.size + 1):
        nodo = demo_treap.kth(k)
        print(f"{k}-ésimo elemento: {nodo.key}")

def demo_interval_tree_queries():
    """
    Demostración adicional de consultas en el árbol de intervalos.
    Inserta una serie de intervalos y realiza varias consultas de solapamiento.
    """
    print("\n Demostración de consultas en árbol de intervalos")
    demo_itree = IntervalTree()
    intervalos_demo = [(1, 3), (2, 6), (4, 7), (5, 9), (8, 10)]
    for intr in intervalos_demo:
        demo_itree.insert(intr[0], intr[1])
    print("Árbol de Intervalos (inorden):")
    for elem in demo_itree.inorder():
        print(elem)
    consultas = [(2, 5), (6, 8), (0, 2)]
    for consulta in consultas:
        solapamientos = demo_itree.search_overlaps(consulta[0], consulta[1])
        print(f"Intervalos que se solapan con {consulta}: {solapamientos}")

# Ejecutar las demostraciones adicionales
demo_treap_operations()
demo_interval_tree_queries()


### Pruebas

Se define una serie de pruebas unitarias utilizando el framework `unittest` para validar el correcto funcionamiento de dos estructuras de datos: el treap y el árbol de intervalos. Se crean dos clases de test, `TestTreap` y `TestIntervalTree`, cada una encargada de verificar diferentes aspectos de sus respectivas estructuras.

En la clase `TestTreap` se realizan varias pruebas. En el método `setUp()`, que se ejecuta antes de cada test, se instancia un treap vacío para garantizar un entorno limpio en cada ejecución. La función `test_insert_and_search()` inserta un conjunto de claves (50, 30, 70, 20, 40, 60 y 80) en el treap y posteriormente utiliza la función de búsqueda para confirmar que cada clave ha sido insertada correctamente. Se comprueba que el nodo encontrado no sea `None` y que la clave del nodo coincida exactamente con la insertada. 

En `test_delete()` se vuelve a insertar el mismo conjunto de claves, se elimina la clave 70 y se verifica que, al buscarla de nuevo, el resultado sea `None`, asegurando que la eliminación se realizó de forma exitosa. La función `test_kth_element()` evalúa el método kth, el cual debe retornar el k-ésimo elemento en orden inorden. Se inserta el conjunto de claves, se ordenan y se recorre la lista comprobando que, para cada posición `k`, el nodo retornado tenga la clave correspondiente según la lista ordenada.

Por su parte, en la clase `TestIntervalTree` se validan las operaciones del árbol de intervalos. El método `setUp()` inicializa un árbol de intervalos vacío. En `test_insert_intervals()` se insertan varios intervalos; posteriormente se realiza un recorrido inorden del árbol, extrayéndose el valor `low` de cada nodo y comprobando que estos estén ordenados, lo cual verifica el mantenimiento de la propiedad de orden en el árbol. Finalmente, en `test_search_overlaps()` se insertan intervalos y se realiza una consulta para obtener aquellos que se solapan con el intervalo `[14,16]`, esperando que se encuentre al menos un intervalo superpuesto.

La ejecución de las pruebas se realiza mediante `unittest.main()`, configurado para ejecutarse en un entorno interactivo sin salir del mismo.

In [ ]:
import unittest

class TestTreap(unittest.TestCase):
    def setUp(self):
        # Se inicializa un treap vacío para cada test.
        self.treap = Treap()

    def test_insert_and_search(self):
        # Inserta un conjunto de claves y verifica que puedan ser buscadas.
        claves = [50, 30, 70, 20, 40, 60, 80]
        for clave in claves:
            self.treap.insert(clave)
        for clave in claves:
            node = self.treap.search(clave)
            self.assertIsNotNone(node, f"La clave {clave} debe estar presente en el treap.")
            self.assertEqual(node.key, clave)
    
    def test_delete(self):
        # Inserta claves y elimina una de ellas, luego verifica que ya no se encuentre.
        claves = [50, 30, 70, 20, 40, 60, 80]
        for clave in claves:
            self.treap.insert(clave)
        self.treap.delete(70)
        self.assertIsNone(self.treap.search(70), "La clave 70 debe haber sido eliminada")
    
    def test_kth_element(self):
        # Inserta claves, ordena la lista y verifica que kth (en recorrido inorden) retorne el elemento correcto.
        claves = [50, 30, 70, 20, 40, 60, 80]
        for clave in claves:
            self.treap.insert(clave)
        sorted_keys = sorted(claves)
        for k in range(1, len(claves)+1):
            kth_node = self.treap.kth(k)
            self.assertIsNotNone(kth_node)
            self.assertEqual(kth_node.key, sorted_keys[k-1])

class TestIntervalTree(unittest.TestCase):
    def setUp(self):
        # Se inicializa un árbol de intervalos vacío para cada test.
        self.tree = IntervalTree()

    def test_insert_intervals(self):
        # Inserta intervalos y verifica que, al hacer recorrido inorden, las claves 'low' estén ordenadas.
        intervals = [(15, 20), (10, 30), (17, 19), (5, 20), (12, 15), (30, 40)]
        for low, high in intervals:
            self.tree.insert(low, high)
        inorder_intervals = self.tree.inorder()
        low_values = [low for low, _, _ in inorder_intervals]
        self.assertEqual(low_values, sorted(low_values))

    def test_search_overlaps(self):
        # Inserta intervalos y consulta los que se solapan con el intervalo [14,16].
        intervals = [(15, 20), (10, 30), (17, 19), (5, 20), (12, 15), (30, 40)]
        for low, high in intervals:
            self.tree.insert(low, high)
        overlaps = self.tree.search_overlaps(14, 16)
        # Se espera que encuentre al menos un intervalo superpuesto.
        self.assertTrue(len(overlaps) > 0, "Debe encontrar al menos un intervalo que solape con [14,16]")

# Ejecutar los tests sin salir de Jupyter Notebook.
unittest.main(argv=[''], verbosity=2, exit=False)


### Profiling

Se utiliza el módulo `cProfile` de Python para medir el rendimiento de operaciones realizadas sobre dos estructuras de datos: el treap y el árbol de intervalos. Se definen dos funciones específicas, una para cada estructura, que ejecutan un conjunto de operaciones y finalmente se perfilan utilizando `cProfile.run()`.

En la función **profile_treap_operations()** se realizan las siguientes acciones en un treap:

1. **Inserción masiva**:  
   Se generan 1000 claves numéricas (de 0 a 999) que son barajadas aleatoriamente. Cada clave se inserta en el treap mediante el método insert(). Esta operación evalúa el rendimiento de la inserción en el treap, donde la estructura del árbol se ajusta para mantener las propiedades de orden y heap.

2. **Eliminación periódica**:  
   Tras la inserción, se eliminan claves cada décimo elemento (usando `slicing keys[::10]`). La eliminación implica la reestructuración del treap mediante la fusión (merge) de los subárboles, lo que impacta en el desempeño del algoritmo.

3. **Búsqueda del k-ésimo elemento**:  
   Una vez realizadas las inserciones y eliminaciones, se recorren todas las posiciones posibles (del 1 al tamaño del treap) utilizando el método `kth()`, que encuentra el k-ésimo nodo en orden inorden. Esta operación evalúa la eficiencia en la búsqueda por posición, que es dependiente de la estructura interna del treap.

Finalmente, la función retorna el recorrido inorden del treap, lo que permite conocer el estado final de la estructura.

En la función **profile_interval_tree_operations()** se realizan operaciones similares pero aplicadas a un árbol de intervalos:

1. **Inserción de intervalos**:  
   Se generan 1000 intervalos, cada uno con una longitud de 10 (por ejemplo, `(i, i+10)` para `i` de 0 a 999) y se insertan en el árbol de intervalos. Durante la inserción, cada nodo se ubica según el valor low y se actualiza la información del valor máximo en el subárbol (max), lo que es fundamental para las consultas posteriores.

2. **Consultas de solapamiento**:  
   Se ejecutan consultas de intersección cada 50 unidades (desde 0 hasta 1000), buscando intervalos que se superpongan con un rango de 5 unidades (`i a i+5`). Estas consultas examinan la estructura del árbol y aprovechan la información del atributo max para omitir búsquedas en subárboles que no puedan contener intervalos solapados.

Al final, la función retorna el recorrido inorden del árbol, mostrando la organización de los intervalos según su límite inferior.

El comando **cProfile.run()** se utiliza para ejecutar cada función y generar un informe que incluye métricas como el tiempo total empleado, el número de llamadas a cada función, y el tiempo medio por llamada. Esto facilita el análisis y optimización del rendimiento, detectando posibles cuellos de botella en las operaciones de inserción, eliminación, búsqueda y consulta dentro de ambas estructuras de datos.

In [ ]:
import cProfile
import random

def profile_treap_operations():
    """
    Función para realizar operaciones sobre el treap y medir su rendimiento:
      - Inserta 1000 claves aleatorias.
      - Elimina algunas claves (cada décima).
      - Realiza búsquedas del k-ésimo elemento en el recorrido inorden.
    """
    treap = Treap()
    # Generar 1000 claves y barajarlas
    keys = list(range(1000))
    random.shuffle(keys)
    for key in keys:
        treap.insert(key)
    
    # Elimina claves cada 10 elementos
    for key in keys[::10]:
        treap.delete(key)
    
    # Ejecuta búsquedas kth para cada posición válida
    if treap.root:
        for i in range(1, treap.root.size + 1):
            _ = treap.kth(i)
    
    return treap.inorder()

def profile_interval_tree_operations():
    """
    Función para realizar operaciones sobre el árbol de intervalos y medir su rendimiento:
      - Inserta 1000 intervalos (cada uno con longitud 10).
      - Realiza consultas de solapamiento cada 50 posiciones.
    """
    tree = IntervalTree()
    # Generar 1000 intervalos: cada intervalo es de la forma (i, i+10)
    intervals = [(i, i+10) for i in range(1000)]
    for low, high in intervals:
        tree.insert(low, high)
    
    # Ejecuta consultas de solapamiento cada 50 unidades
    for i in range(0, 1000, 50):
        _ = tree.search_overlaps(i, i+5)
    
    return tree.inorder()

print("Perfilando operaciones del treap:")
cProfile.run('profile_treap_operations()')

print("\nPerfilando operaciones del árbol de intervalos:")
cProfile.run('profile_interval_tree_operations()')

### Ejercicios

1. **Refactorización del Código:**  
   - Separa la implementación del treap y del árbol de intervalos en módulos independientes.  
   - Implementa logging (por ejemplo, con la librería `logging`) para registrar eventos importantes en las operaciones (inserción, eliminación, búsquedas, etc.) sin utilizar `print`, lo que facilitará futuras revisiones de rendimiento o debug.

2. **Comprobación de casos extremos y modularización:**  
   - Escribe funciones adicionales para validar la consistencia del árbol tras múltiples operaciones.
   - Agrega pruebas unitarias más extensas (por ejemplo, pruebas de estrés o de rendimiento en condiciones límite).

3. **Utilización de cProfile y pstats:**  
   - Modifica el script de profiling para que, además de ejecutar `cProfile.run()`, guarde los resultados en un archivo.  
   - Emplea el módulo `pstats` para ordenar y filtrar la información (por tiempo de ejecución total, número de llamadas, etc.).  
   - Ejemplo:  
     ```python
     import cProfile, pstats
     profiler = cProfile.Profile()
     profiler.run('profile_treap_operations()')
     stats = pstats.Stats(profiler)
     stats.sort_stats('cumtime').dump_stats('treap_profile.stats')
     ```

4. **Almacenamiento en CSV:**  
   - Desarrolla una función que extraiga del objeto pstats la información relevante (nombre de la función, número de llamadas, tiempo acumulado, etc.) y la guarde en un archivo CSV para su posterior análisis con pandas.

5. **Procesamiento con Pandas:**  
   - Lee el archivo CSV generado en el ejercicio anterior utilizando pandas.  
   - Organiza los datos en un DataFrame, facilitando la selección de las funciones con mayor costo en tiempo.
   
6. **Gráficos de rendimiento:**  
   - Usa matplotlib para generar gráficos de barras o de líneas que muestren, por ejemplo, el tiempo acumulado de las funciones más costosas.  
   - Crea gráficos comparativos entre diferentes operaciones del treap y el árbol de intervalos.  
   - Ejemplo de código:
     ```python
     import pandas as pd
     import matplotlib.pyplot as plt
     
     df = pd.read_csv('profiling_results.csv')
     df_top = df.nlargest(10, 'cumtime')
     plt.figure(figsize=(10, 6))
     plt.barh(df_top['func_name'], df_top['cumtime'])
     plt.xlabel('Tiempo acumulado (segundos)')
     plt.title('Top 10 Funciones por Tiempo de Ejecución')
     plt.gca().invert_yaxis()
     plt.show()
     ```

7. **Análisis de memoria:**  
   - Investiga y aplica la biblioteca `memory_profiler` para evaluar el consumo de memoria durante las operaciones del treap y el árbol de intervalos.

8. **Optimización automática:**  
   - Explora la posibilidad de aplicar decoradores para medir de forma transparente el tiempo de ejecución y la memoria utilizada en cada función crítica.

In [ ]:
### Tus respuestas